# Práctica 1: Aprendizaje Automático
## Parte A: Evaluación de modelos de Aprendizaje Automático
Hecho por:
- Jaime Benedí
- Miguel Sevilla

En esta parte vamos a utilizar la librería [surprise](https://surprise.readthedocs.io/en/stable/) que implementa algoritmos y otros procedimientos para desarrollar y evaluar algoritmos de sistemas de recomendación. Asegúrate de tenerla instalada antes de empezar a hacer las tareas.

In [1]:
!pip install numpy
!pip install scikit-surprise


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


Trabajaremos sobre el dataset de [MovieLens de 100K](https://grouplens.org/datasets/movielens/100k/). MovieLens es un dataset muy usado en el desarrollo y evaluación de sistemas de recomendación, y el dataset de 100k contiene 100K interacciones entre usuarios y productos, almacenando qué usuario ha puntuado qué película y con qué rating.

## Objetivos
1. Cargar MovieLens de 100k.

2. Divide el dataset en una partición simple, donde el conjunto de evaluación sea el 75% de las interacciones y el resto forme parte del conjunto de evaluación. Aquí tenéis que pasarle random_state = SEED a la función.

3. Vamos a evaluar distintos algoritmos de recomendación:
   1. Filtrado colaborativo basado en vecinos (KNNBasic en esta [librería](https://surprise.readthedocs.io/en/stable/knn_inspired.html)) tanto basado en usuarios como enproductos. Utilizaremos la métrica de similitud de Pearson.
   2. Filtrado colaborativo basado en modelos usando [factorización de matrices](https://surprise.readthedocs.io/en/stable/matrix_factorization.html), usando los algoritmos de SVD y NMF.

4. Cada uno de estos algoritmos se entrenarán con el conjunto de
entrenamiento. Aquí tenéis que pasarle random_state = SEED a cada uno de los modelos.

5. Después se obtendrán las predicciones que todos los algoritmos obtienen para el conjunto de evaluación. Muestra el resultado de 5 predicciones e interpreta los resultados.

6. Crea una tabla con los valores que se obtienen para las métricas de evaluación RMSE, precision@k, recall@k, y NDCG, k es el tamaño de la lista de recomendación y será k = 10. Surprise solo implementa RMSE. Las demás las podéis encontrar en sklearn usando precision_score, recall_score, ndcg_score.
   1. **IMPORTANTE**: solo las películas cuyo rating sea superior a 4 serán consideradas relevantes (incluyendo en la lista de películas recomendadas por el modelo).

7. Explica cada uno de los resultados obtenidos y qué significado tienen. Determina cuál podría ser el mejor método recomendador a utilizar.

## Antes de empezar...
Determina un valor para una variable SEED (el que sea). Esta variable se la vamos a pasar a los modelos para que cada vez que se ejecute el código salgan los mismos resultados. De esta forma os aseguráis de que los resultados que os salgan serán los mismos que me salgan a mí al ejecutar el código. Si no hacéis esto, vuestro análisis puede no tener ningún sentido en mi ejecución y os arriesgáis al no apto.

In [12]:
SEED = 42

In [13]:
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

In [14]:
# 1 -> Cargar el dataset
data = Dataset.load_builtin("ml-100k")

In [15]:
# 2 -> Dividir el dataset: 75% conjunto de evaluación (Posible errata -> ¿Entrenamiento?)
# pasarle random_state = SEED a la función -> facilita la reproducibilidad de los experimentos

from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25,random_state = SEED)

In [23]:
# 3.1 Algoritmo de recomendación: KNNBasic
from surprise import Dataset, KNNBasic

# Retrieve the trainset.
trainset = data.build_full_trainset()
# Build an algorithm, and train it.
algo = KNNBasic(random_state = SEED)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [24]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.06   {'actual_k': 40, 'was_impossible': False}


In [25]:
# 3.2 Algoritmo de recomendación: Factorización de matrices -> usando los algoritmos de SVD y NMF
from surprise import accuracy

algo2 = SVD(random_state = SEED)
# Train the algorithm on the trainset, and predict ratings for the testset
algo2.fit(trainset)
predictions = algo2.test(testset)
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.6757


0.6757273833579454

In [29]:
# 5. Muestra el resultado de 5 predicciones e interpreta los resultados
cross_validate(algo2, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9390  0.9383  0.9314  0.9343  0.9415  0.9369  0.0036  
MAE (testset)     0.7382  0.7403  0.7329  0.7368  0.7436  0.7384  0.0035  
Fit time          0.41    0.40    0.41    0.42    0.42    0.41    0.01    
Test time         0.04    0.10    0.04    0.04    0.09    0.06    0.03    


{'test_rmse': array([0.93895242, 0.93826423, 0.93143621, 0.93431584, 0.9415186 ]),
 'test_mae': array([0.73823621, 0.74026092, 0.73293454, 0.73684947, 0.7436165 ]),
 'fit_time': (0.4092679023742676,
  0.3960611820220947,
  0.4127209186553955,
  0.41632699966430664,
  0.42296600341796875),
 'test_time': (0.03909921646118164,
  0.09842205047607422,
  0.04078793525695801,
  0.04114389419555664,
  0.09339499473571777)}